## Install package and download required data files

In [ ]:
# Download ADSO ontology file
!wget "https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl"

--2023-03-08 10:14:38--  https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68571 (67K) [text/plain]
Saving to: ‘ADSOv1.0.3.owl.1’

ADSOv1.0.3.owl.1    100%[===================>]  66.96K  --.-KB/s    in 0.01s   

2023-03-08 10:14:38 (4.50 MB/s) - ‘ADSOv1.0.3.owl.1’ saved [68571/68571]



In [ ]:
!pip install rdflib spacy sentence-transformers huggingface-hub transformers gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load required pre-trained models




In [ ]:
# Download Wordnet dependencies
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

# Get a list of English stop words
stop_words = stopwords.words('english')

In [ ]:
from sentence_transformers import SentenceTransformer, util
from huggingface_hub import from_pretrained_keras
import numpy as np
import transformers
import tensorflow as tf
import joblib

# POS tagger
pos_tagger = joblib.load("/content/pos-tagger.joblib")

# Text similarity model
tsm = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2');

# Semantic similarity model
ssm = SentenceTransformer('Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt')

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For mor

In [ ]:
# Helper functions to define the properties of the pos-tagger model
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

## Data loader component

In here we are going to retreive the all type of entities/nodes from the ontology knowledgebase and store in seperate arrays.



In [ ]:
# Load downloaded ontology graph
from rdflib import Graph, Namespace, Literal, RDF, URIRef

g = Graph()
g.parse("/content/ADSOv1.0.3.owl")

<Graph identifier=N3db6d694e9d7452c83e51e517255b14f (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Get the URIRef for given keyword
def get_ref(keyword):
    return URIRef("https://ontology.drpawspaw.com/"+keyword)

In [ ]:
# Get the entity 'text' from the URIRef
def get_text_from_uri(uri):
    for s,p,o in g:
        if s == uri and p == URIRef("https://ontology.drpawspaw.com/text"):
            return o
    return None

In [ ]:
print(get_text_from_uri(URIRef('https://ontology.drpawspaw.com/ADSO0000000083')))

Distempter


In [ ]:
# Get list of symptoms from disease URI
def get_symptoms_from_disease_uri(uri):
  symptoms = []
  for s,p,o in g:
    if s == uri and p == URIRef("https://ontology.drpawspaw.com/hasSymptom"):
      symptoms.append(o.toPython())
  return symptoms

In [ ]:
get_symptoms_from_disease_uri(URIRef('https://ontology.drpawspaw.com/ADSO0000000083'))

['https://ontology.drpawspaw.com/ADSO0000000039',
 'https://ontology.drpawspaw.com/ADSO0000000036',
 'https://ontology.drpawspaw.com/ADSO0000000031',
 'https://ontology.drpawspaw.com/ADSO0000000093',
 'https://ontology.drpawspaw.com/ADSO0000000037',
 'https://ontology.drpawspaw.com/ADSO0000000007',
 'https://ontology.drpawspaw.com/ADSO0000000026',
 'https://ontology.drpawspaw.com/ADSO0000000032',
 'https://ontology.drpawspaw.com/ADSO0000000033',
 'https://ontology.drpawspaw.com/ADSO0000000030',
 'https://ontology.drpawspaw.com/ADSO0000000008',
 'https://ontology.drpawspaw.com/ADSO0000000040',
 'https://ontology.drpawspaw.com/ADSO0000000023',
 'https://ontology.drpawspaw.com/ADSO0000000038',
 'https://ontology.drpawspaw.com/ADSO0000000027',
 'https://ontology.drpawspaw.com/ADSO0000000017',
 'https://ontology.drpawspaw.com/ADSO0000000034',
 'https://ontology.drpawspaw.com/ADSO0000000028',
 'https://ontology.drpawspaw.com/ADSO0000000009',
 'https://ontology.drpawspaw.com/ADSO0000000005']

In [ ]:
# Get URI from text
def get_uri_from_text(text):
  for s, p, o in g:
    if p == URIRef("https://ontology.drpawspaw.com/text"):
      if o.toPython().lower() == text.lower():
        return s
  return None

In [ ]:
get_uri_from_text("fever")

rdflib.term.URIRef('https://ontology.drpawspaw.com/ADSO0000000005')

In [ ]:
# Collect named entities from ontology
diseases = []
symptoms = []
synonyms = []

for s,p,o in g:
    if p == get_ref("hasDisease"):
        try:
            dis = get_text_from_uri(o).toPython()
            diseases.append(dis)
        except:
            print(type(o), o)
    if p == get_ref("hasSymptom"):
        try:
            sym = get_text_from_uri(o).toPython()
            symptoms.append(sym)
        except:
            print(type(o), o)
    if p == get_ref("hasSynonym"):
        try:
            syn = get_text_from_uri(o).toPython()
            synonyms.append(syn)
        except:
            print(type(o), o)

<class 'rdflib.term.URIRef'> https://ontology.drpawspaw.com/ADSO0000000110


In [ ]:
# List of symptoms and their synonyms
symp_syn = []

# Check the keyword already exist or not in collection
def is_exist_ss(keyword):
    for _,k,_ in symp_syn:
        if k == keyword:
            return True
            
# Apped all the symptoms and their synonyms
for s,p,o in g: # subject, predicate, object
    # filter synonyms
    if p == get_ref("hasSymptom"):
        try:
            x = get_text_from_uri(o).toPython()
        except:
            continue
        curr_symp_syn = []
        for s1, p1, o1 in g:
            # filter synonym for above "o" entity
            if s1 == o and p1 == get_ref("hasSynonym"):
                try:
                    y = get_text_from_uri(o1).toPython()
                    curr_symp_syn.append(y)                    
                except:
                    continue
        # validate to add only one entry
        if not is_exist_ss(x):
            try:
                idx = o.toPython()
                symp_syn.append((idx, x, curr_symp_syn))
            except:
                continue

In [ ]:
symp_syn[:5]

[('https://ontology.drpawspaw.com/ADSO0000000012', 'Stroke', []),
 ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis', []),
 ('https://ontology.drpawspaw.com/ADSO0000000042', 'Running Nose', []),
 ('https://ontology.drpawspaw.com/ADSO0000000026',
  'Nasal Discharge',
  ['Nasal Whistling', 'Nasal Wheezing']),
 ('https://ontology.drpawspaw.com/ADSO0000000074', 'Skin Scaling', [])]

In [ ]:
len(symp_syn)

79

## Text similarity

In here we are going to calculate the similarity of the words using pretrained model from Hugging Face => Text Similarity - To get the similar words for identified named entities (https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2, https://huggingface.co/tasks/sentence-similarity)

In [ ]:
# Generate synonyms for the existing symptoms and update the list of synonyms
from nltk.corpus import wordnet

def generate_synonyms(uri, keyword, syns):
    for synonym in wordnet.synsets(keyword):
        for item in synonym.lemmas():
            if keyword != synonym.name() and len(synonym.lemma_names()) > 1:
                syns.append(item.name())
    return (uri, keyword, syns)

In [ ]:
# for idx,(uri,keyword,syns) in enumerate(symp_syn):
#   symp_syn[idx] = generate_synonyms(uri,keyword,syns)

In [ ]:
print(len(symp_syn))
print(symp_syn[:5])

79
[('https://ontology.drpawspaw.com/ADSO0000000012', 'Stroke', []), ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis', []), ('https://ontology.drpawspaw.com/ADSO0000000042', 'Running Nose', []), ('https://ontology.drpawspaw.com/ADSO0000000026', 'Nasal Discharge', ['Nasal Whistling', 'Nasal Wheezing']), ('https://ontology.drpawspaw.com/ADSO0000000074', 'Skin Scaling', [])]


In [ ]:
# Exapand the all entities
expanded_symp = []

def is_exist_es(keyword):
    for idx,word in expanded_symp:
        if keyword == word:
            return True

for idx,word,syns in symp_syn:
    for s in syns:
        if not is_exist_es(s):
            expanded_symp.append((idx, s))
    if not is_exist_es(word):
        expanded_symp.append((idx, word))

In [ ]:
print(len(expanded_symp))
print(expanded_symp[:5])

101
[('https://ontology.drpawspaw.com/ADSO0000000012', 'Stroke'), ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis'), ('https://ontology.drpawspaw.com/ADSO0000000042', 'Running Nose'), ('https://ontology.drpawspaw.com/ADSO0000000026', 'Nasal Whistling'), ('https://ontology.drpawspaw.com/ADSO0000000026', 'Nasal Wheezing')]


In [ ]:
# Calculate the similariity
# Here "tsm" means the text-similarity-model, that we loaded in "Load required pre-trained models"
# "get_similarity" accepts the URI, identified named entity, entity from ontology
def get_similarity(idx, word, ne): # Return a tuple, contain URI, node, similarity_score
    embedding_1 = tsm.encode(ne, convert_to_tensor=True)
    embedding_2 = tsm.encode(word, convert_to_tensor=True)
    return (idx, word, util.pytorch_cos_sim(embedding_1, embedding_2))

In [ ]:
# At the current implementation, we only get first five entities
# "entity" - identified named entity, "nodes" - named entities in the onotology (in format of [(idx, text)])
def get_most_similarity_entities(entity, nodes):
  scores = map(lambda e: get_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=lambda x:x[2], reverse=True)[:5]

In [ ]:
# Tryout the implementation
identified_ne = ["fever", "lethargy"]

# Get the top five entities from the text similarity, accroding to the cosine similarity score
get_most_similarity_entities(identified_ne[0], expanded_symp)

[('https://ontology.drpawspaw.com/ADSO0000000005', 'Fever', tensor([[1.]])),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Fever',
  tensor([[0.8284]])),
 ('https://ontology.drpawspaw.com/ADSO0000000032',
  'Pneumonia',
  tensor([[0.4948]])),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Temperature',
  tensor([[0.4413]])),
 ('https://ontology.drpawspaw.com/ADSO0000000008',
  'Vomiting',
  tensor([[0.4372]]))]

## Semantic Similarity

In here we are going to calculate the semantic similarity of the words using pretrained model from HuggingFace => Text/Paragraph and Sentence similarity - This model especially designed to check the semantic similarity of biological related things (https://huggingface.co/pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb)

```
Example:
- Percentage similarity between 'fever' and 'cough' is 72.12%
- Percentage similarity between 'fever' and 'high body temperature' is 76.15%
```

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the semantic similarity of the identified entity and nodes in ontology
# "calculate_semantic_similarity" functions accpets, the URI, text and identified entity
def calculate_semantic_similarity(uri, ctx, entity):
    sent_embed = ssm.encode([entity, ctx])
    # Compute cosine similarity between the two sentence embeddings
    similarity = cosine_similarity([sent_embed[0]], [sent_embed[1]])[0][0]
    # Convert cosine similarity to percentage similarity
    percentage_similarity = round((similarity + 1) / 2 * 100, 2)
    return (uri, ctx, percentage_similarity)

In [ ]:
# "get_highest_ctx_similarity" function return the highest context similarity as tuple (URI, Word, ctx_similarity)
def get_highest_ctx_similarity(entity, nodes):
  scores = map(lambda e: calculate_semantic_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=lambda x:x[2], reverse=True)[:1]

In [ ]:
ine = "fever"
candidates = get_most_similarity_entities(ine, expanded_symp) # Candidates from text-similarity
get_highest_ctx_similarity(ine, candidates)

[('https://ontology.drpawspaw.com/ADSO0000000005', 'Fever', 88.27)]

## Query data

In [ ]:
# This function return SPARQL query, that able to get the disease from symptoms
def build_query(symptoms):
  return """
    PREFIX adso: <https://ontology.drpawspaw.com/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?diseaseName
    WHERE {{
        {sym_query}
        ?diseaseUri adso:text ?diseaseName .
    }}
    """.format(sym_query="\n".join(
        list(
            map(lambda e: "?diseaseUri adso:hasSymptom adso:{uri} .".format(uri=e),
                list(map(lambda e: e.split("/")[3], symptoms))))))

In [ ]:
q = build_query(["https://ontology.drpawspaw.com/ADSO0000000005",
    "https://ontology.drpawspaw.com/ADSO0000000006",
    "https://ontology.drpawspaw.com/ADSO0000000007"])

for output in g.query(q):
  print(output)

(rdflib.term.Literal('Babesiosis', lang='en'),)


## Implementation tryout

### Preprocessing inputs

In [ ]:
# Preprocessing (remove stop words)
# user_input = "My dog has been vomiting has Limb Swelling and has diarrhea"
user_input = "My dog has been vomiting and has diarrhea "

# Tokenize the sentence
tokens = nltk.word_tokenize(user_input)

# Remove stop words, (stopwords are getting fromt the NLTK library)
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

# Filtered tokens as sentence
filtered_user_input = " ".join(filtered_tokens)

In [ ]:
filtered_user_input

'dog vomiting diarrhea'

In [ ]:
from nltk import word_tokenize

def pos_tag(sentence):
    tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

annotated_entites = [en for en in list(
      pos_tag(word_tokenize(filtered_user_input)) # Filter entities annotate as "SYMPTOM"
    ) if en[1] == "SYMPTOM"]

In [ ]:
annotated_entites

[('vomiting', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')]

### Processing entity linker component

In [ ]:
link_entity = []

for entity in annotated_entites:
  current_entity_candidates = get_most_similarity_entities(entity[0], expanded_symp)
  link_entity.append(get_highest_ctx_similarity(entity[0], current_entity_candidates))

In [ ]:
link_entity

[[('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', 98.06)],
 [('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', 97.38)]]

### Query symptoms to get the disease

In [ ]:
entities = list(map(lambda x: x[0][0], link_entity))

In [ ]:
entities

['https://ontology.drpawspaw.com/ADSO0000000008',
 'https://ontology.drpawspaw.com/ADSO0000000009']

In [ ]:
curr_query = build_query(entities)

In [ ]:
curr_query

'\n    PREFIX adso: <https://ontology.drpawspaw.com/>\n    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n    SELECT ?diseaseName\n    WHERE {\n        ?diseaseUri adso:hasSymptom adso:ADSO0000000008 .\n?diseaseUri adso:hasSymptom adso:ADSO0000000009 .\n        ?diseaseUri adso:text ?diseaseName .\n    }\n    '

In [ ]:
for rq in g.query(curr_query):
  print(rq)

(rdflib.term.Literal('Babesiosis', lang='en'),)
(rdflib.term.Literal('Distempter', lang='en'),)


In [ ]:
WORD_JOIN_CHAR = " "; # Use to join the words in an array

from nltk import word_tokenize

def pos_tag(sentence):
    tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

def entity_linker(sentence):
  etq = [] # Entities to query
  annotated_entites = [en for en in list(
      pos_tag(word_tokenize(sentence)) # Filter entities annotate as "SYMPTOM"
    ) if en[1] == "SYMPTOM"]
  print("Identified named entities => ", annotated_entites, '\n')
  if len(annotated_entites) == 0:
    return []
  for e in annotated_entites:
    current_entity_candidates = get_most_similarity_entities(e[0], expanded_symp)
    print("Candidates => ",e, current_entity_candidates)
    etq.append(get_highest_ctx_similarity(e, current_entity_candidates))
  print("\nNamed Entity => ", etq)
  current_query = build_query(list(map(lambda x: x[0][0], etq)))
  print("\nQuery => ", current_query)
  return [rq for rq in g.query(current_query)]

In [ ]:
entity_linker("My dog has been vomiting has Limb Swelling and has diarrhea")

Identified named entities =>  [('vomiting', 'SYMPTOM'), ('Swelling', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')] 

Candidates =>  ('vomiting', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', tensor([[1.0000]])), ('https://ontology.drpawspaw.com/ADSO0000000057', 'Nausea', tensor([[0.7699]])), ('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', tensor([[0.6183]])), ('https://ontology.drpawspaw.com/ADSO0000000066', 'Eating Poison', tensor([[0.5464]])), ('https://ontology.drpawspaw.com/ADSO0000000007', 'coughing', tensor([[0.5324]]))]
Candidates =>  ('Swelling', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000048', 'Swelling Limbs', tensor([[0.7699]])), ('https://ontology.drpawspaw.com/ADSO0000000014', 'Limb Swelling', tensor([[0.7656]])), ('https://ontology.drpawspaw.com/ADSO0000000100', 'Swelling of Ear Canals', tensor([[0.5785]])), ('https://ontology.drpawspaw.com/ADSO0000000015', 'Heavy Pain', tensor([[0.5241]])), ('https://ontology.drpawspaw.com/AD

[]

In [ ]:
from itertools import chain

def make_prediction(sentence):
  prediction = entity_linker(sentence)
  # prediction = [(rdflib.term.Literal('Babesiosis', lang='en'),), (rdflib.term.Literal('Distempter', lang='en'),)]
  # If there are multiple predictions from the given symptoms
  if len(prediction) > 1:
    # Retrieve the URI for above prediction
    print("prediction", prediction)
    duris = list(map(lambda x: get_uri_from_text(x[0].toPython()).toPython(), prediction))
    # Get the all unique symptoms for predicted diseases
    symptoms_uris = list(map(lambda x: get_symptoms_from_disease_uri(URIRef(x)), list(duris)))
    print("symptoms_uris", symptoms_uris)
    # Convert back to symptoms into readable format
    symptom_suggestions = list(map(lambda x: get_text_from_uri(URIRef(x)).toPython(), list(chain.from_iterable(symptoms_uris))))
    print("symptom_suggestions", symptom_suggestions)
    # Return symptoms suggestions for the better prediction
    return {
      "request": sentence,
      "result_type": "SUGGESTION",
      "symptom_suggestions": list(set(symptom_suggestions)),
      "predicted_disease": None
    }
  # Unable to predict
  if len(prediction) < 1:
    return {
      "request": sentence,
      "result_type": "LIMITATION",
      "symptom_suggestions": None,
      "predicted_disease": None
    }
  # Return the prediction
  result = {
      "request": sentence,
      "result_type": "PREDICTION",
      "symptom_suggestions": None,
      "predicted_disease": prediction[0]
  }
  return result

In [ ]:
import time

start_time = time.time()
print(make_prediction('My dog has been vomiting has Limb Swelling and has diarrhea'))
print("--- Time elaped for single execution: ", time.time()-start_time, " ---")

Identified named entities =>  [('vomiting', 'SYMPTOM'), ('Swelling', 'SYMPTOM'), ('diarrhea', 'SYMPTOM')] 

Candidates =>  ('vomiting', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000008', 'Vomiting', tensor([[1.0000]])), ('https://ontology.drpawspaw.com/ADSO0000000057', 'Nausea', tensor([[0.7699]])), ('https://ontology.drpawspaw.com/ADSO0000000009', 'Diarrhea', tensor([[0.6183]])), ('https://ontology.drpawspaw.com/ADSO0000000066', 'Eating Poison', tensor([[0.5464]])), ('https://ontology.drpawspaw.com/ADSO0000000007', 'coughing', tensor([[0.5324]]))]
Candidates =>  ('Swelling', 'SYMPTOM') [('https://ontology.drpawspaw.com/ADSO0000000048', 'Swelling Limbs', tensor([[0.7699]])), ('https://ontology.drpawspaw.com/ADSO0000000014', 'Limb Swelling', tensor([[0.7656]])), ('https://ontology.drpawspaw.com/ADSO0000000100', 'Swelling of Ear Canals', tensor([[0.5785]])), ('https://ontology.drpawspaw.com/ADSO0000000015', 'Heavy Pain', tensor([[0.5241]])), ('https://ontology.drpawspaw.com/AD

## Evaluation

### Create test data from the ontology and SPARQL

In [ ]:
# Create test data set using the ontology knowledgebase
'''
[((symptom1, symptom2, ...), disease)]
'''
import random

test_data = [] # 1
while len(test_data) < 100:
  get_random_num = random.randint(1, 3)
  stup = []
  for sid in range(get_random_num):
    sidx = random.randint(0, len(symptoms)-1)
    if get_uri_from_text(symptoms[sidx]) == None:
      continue
    try:
      curr_symp = get_uri_from_text(symptoms[sidx]).toPython()
      stup.append(curr_symp)
    except:
      continue
  q = build_query(stup)
  pred = [output for output in g.query(q)]
  if len(pred) == 1:
    test_data.append((stup, [output for output in g.query(q)]))

In [ ]:
test_data[:5]

[(['https://ontology.drpawspaw.com/ADSO0000000027'],
  [(rdflib.term.Literal('Distempter', lang='en'))]),
 (['https://ontology.drpawspaw.com/ADSO0000000040'],
  [(rdflib.term.Literal('Distempter', lang='en'))]),
 (['https://ontology.drpawspaw.com/ADSO0000000033'],
  [(rdflib.term.Literal('Distempter', lang='en'))]),
 (['https://ontology.drpawspaw.com/ADSO0000000098'],
  [(rdflib.term.Literal('Ear Infections', lang='en'))]),
 (['https://ontology.drpawspaw.com/ADSO0000000074'],
  [(rdflib.term.Literal('Demodicosis', lang='en'))])]

### Perform testing

In [ ]:
# Update the entity linker component and remove the pos_tag, since it already evaluated individually
WORD_JOIN_CHAR = " "; # Use to join the words in an array

from nltk import word_tokenize

def pos_tag(sentence):
    tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

def entity_linker_for_eval(identified):
  etq = [] # Entities to query
  for e in identified:
    opt_en = [e, "SYMPTOM"]
    current_entity_candidates = get_most_similarity_entities(opt_en[0], expanded_symp)
    etq.append(get_highest_ctx_similarity(opt_en, current_entity_candidates))
  current_query = build_query(list(map(lambda x: x[0][0], etq)))
  return [rq for rq in g.query(current_query)]

In [ ]:
test_data[2]

(['https://ontology.drpawspaw.com/ADSO0000000033'],
 [(rdflib.term.Literal('Distempter', lang='en'))])

In [ ]:
# Using the module, predict the disease from the sentence
# Skip the first element of the array, since it's the headers
"""
 (
   [
     'https://ontology.drpawspaw.com/ADSO0000000020'
   ],
   [
     (rdflib.term.Literal('Parvo', lang='en')),
     (rdflib.term.Literal('Pneumonia', lang='en'))
   ]
  ),
"""

test_result = [] # (idx, symptoms ,linker_pred, actual_pred)
for idx, sentence in enumerate(test_data[1:]):
  print("\n---------- Running test case #{test_id} ----------".format(test_id=idx+1))
  curr_symps = list(map(lambda x: get_text_from_uri(URIRef(x)).toPython(), sentence[0]))
  curr_pred = [p[0].toPython() for p in entity_linker_for_eval(curr_symps)]
  test_result.append((idx, " ".join(curr_symps), curr_pred, [p[0].toPython() for p in sentence[1]]))


---------- Running test case #1 ----------

---------- Running test case #2 ----------

---------- Running test case #3 ----------

---------- Running test case #4 ----------

---------- Running test case #5 ----------

---------- Running test case #6 ----------

---------- Running test case #7 ----------

---------- Running test case #8 ----------

---------- Running test case #9 ----------

---------- Running test case #10 ----------

---------- Running test case #11 ----------

---------- Running test case #12 ----------

---------- Running test case #13 ----------

---------- Running test case #14 ----------

---------- Running test case #15 ----------

---------- Running test case #16 ----------

---------- Running test case #17 ----------

---------- Running test case #18 ----------

---------- Running test case #19 ----------

---------- Running test case #20 ----------

---------- Running test case #21 ----------

---------- Running test case #22 ----------

---------- Running

In [ ]:
# Calculate the accuracy/classification report
def contain_common_elements(arr1, arr2):
  arr1.sort()
  arr2.sort()
  return arr1 == arr2

incorrect_count = 0
for ts in test_result:
  if not contain_common_elements(ts[2], ts[3]):
    incorrect_count += 1

# Accuracy of model = 1 - (number_incorrect_sample / number_of_test_samples)
accuracy = 1 - (incorrect_count / len(test_result))
print('Accuracy of the module is ', accuracy)

Accuracy of the module is  0.76767676767676768
